In [187]:
import numpy as np
from qiskit import IBMQ, Aer, assemble, transpile, execute
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.providers.ibmq import least_busy
from qiskit.quantum_info.operators import Operator, Pauli
from qiskit.algorithms import NumPyEigensolver
from numpy import linalg as LA
from scipy.linalg import expm

In [188]:
def is_unitary(m):
    return np.allclose(np.eye(len(m)), m.dot(m.T.conj()))

In [189]:
coefficients = []
bases = []
with open('LiH-Hamiltonian.txt') as f:
    for line in f:
        x = line.strip().split(" ")
        if (len(line) > 1):
            c = float(x[1])
            c = (c if x[0] == '+' else -c)
            coefficients.append(c)
            bases.append(x[3])
coefficients[:10], bases[:10]


([0.003034656830204855,
  0.003034656830204855,
  0.003034656830204855,
  0.003034656830204855,
  -0.008373361424264817,
  -0.008373361424264817,
  -0.008373361424264817,
  -0.008373361424264817,
  0.00211113766859809,
  0.00211113766859809],
 ['IIIYYIIIYY',
  'IIIXXIIIYY',
  'IIIYYIIIXX',
  'IIIXXIIIXX',
  'YZZZYIIIYY',
  'XZZZXIIIYY',
  'YZZZYIIIXX',
  'XZZZXIIIXX',
  'YZZYIIIIYY',
  'XZZXIIIIYY'])

In [190]:
hamiltonian = np.zeros((2**10, 2**10), dtype=np.cdouble)
# comment this out when stuff actually works
coefficients = [0.5, 0.5]
bases = ['XIIIIIIIII', 'XIIIIIIIII']
numTerms = len(bases)
for c, lbl in zip(coefficients[0:numTerms], bases[0:numTerms]):
    op = Operator(Pauli(label=lbl))
    hamiltonian+=c*op.data

is_unitary(hamiltonian)

True

In [191]:
t = 1
qc = QuantumCircuit(10)
numTerms = 10
for c, b in zip(coefficients[0:numTerms], bases[0:numTerms]):
    # implement e^{-i*c*b*t}
    print(f"Coefficient, {c}")
    print(f"Bases, {b}")
    q = []
    for i, op in enumerate(b):
        if (op == 'X'):
            qc.h(i)
            q.append(i)
        elif (op == 'Z'):
            q.append(i)
        elif (op == 'Y'):
            qc.h(i)
            qc.s(i)
            q.append(i)
        elif (op == 'I'):
            continue
        else:
            print(op, "case not accounted for")
    if (len(q) > 0):
        if (len(q) > 1):
            for i in range(len(q)-1):
                qc.cx(q[i], q[-1])
        qc.rz(2*c*t, q[-1])
        if (len(q) > 1):
            for i in range(len(q)-1):
                qc.cx(q[i], q[-1])
        for i, op in enumerate(b):
            if (op == 'X'):
                qc.h(i)
            elif (op == 'Z'):
                continue
            elif (op == 'Y'):
                qc.sdg(i)
                qc.h(i)
            elif (op == 'I'):
                continue
            else:
                print("case not accounted for")

Coefficient, 0.5
Bases, XIIIIIIIII
Coefficient, 0.5
Bases, XIIIIIIIII


In [192]:
qc.draw()

┌───┐┌───────┐┌───┐┌───┐┌───────┐┌───┐
q_0: ┤ H ├┤ Rz(1) ├┤ H ├┤ H ├┤ Rz(1) ├┤ H ├
     └───┘└───────┘└───┘└───┘└───────┘└───┘
q_1: ──────────────────────────────────────
                                           
q_2: ──────────────────────────────────────
                                           
q_3: ──────────────────────────────────────
                                           
q_4: ──────────────────────────────────────
                                           
q_5: ──────────────────────────────────────
                                           
q_6: ──────────────────────────────────────
                                           
q_7: ──────────────────────────────────────
                                           
q_8: ──────────────────────────────────────
                                           
q_9: ──────────────────────────────────────

In [193]:
backend = Aer.get_backend('unitary_simulator')
job = execute(qc, backend)
result = job.result()
U_circuit = result.get_unitary(qc, decimals=3) 

In [194]:
diff = U_circuit - expm(-1.j * hamiltonian)
diff

Operator([[-0.00030231+0.j   , -0.        -0.841j,  0.        +0.j   ,
           ...,  0.        +0.j   ,  0.        +0.j   ,
            0.        +0.j   ],
          [ 0.        -0.841j, -0.00030231+0.j   ,  0.        +0.j   ,
           ...,  0.        +0.j   ,  0.        +0.j   ,
            0.        +0.j   ],
          [ 0.        +0.j   ,  0.        +0.j   , -0.00030231+0.j   ,
           ...,  0.        +0.j   ,  0.        +0.j   ,
            0.        +0.j   ],
          ...,
          [ 0.        +0.j   ,  0.        +0.j   ,  0.        +0.j   ,
           ..., -0.00030231+0.j   ,  0.        +0.j   ,
            0.        +0.j   ],
          [ 0.        +0.j   ,  0.        +0.j   ,  0.        +0.j   ,
           ...,  0.        +0.j   , -0.00030231+0.j   ,
           -0.        -0.841j],
          [ 0.        +0.j   ,  0.        +0.j   ,  0.        +0.j   ,
           ...,  0.        +0.j   ,  0.        -0.841j,
           -0.00030231+0.j   ]],
         input_dims=(2, 2, 2, 

10 qubits -> 2^10 states -> 2^20 entries ($\approx$ 1,000,000, which is quite reasonable)

In [195]:
# measure the operator's accuracy
data = diff.data
w, v = LA.eig(data)

# measure the error relative to a matrix of 1s for reference.
ones = np.ones(data.shape)
fakeDiff = U_circuit - ones
w_fake, v_fake = LA.eig(fakeDiff.data)

In [196]:
print(max(w))
print(max(w_fake))

(-0.0003023058681385985-0.000470984807896435j)
(0.5400000000003631-0.8410000000001528j)
